# Modelo de reconocimiento de imagenes satelitales con CNN
### Santiago Fandiño Gomez
### Juan Manuel Durán 

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns 
from pathlib import Path

In [7]:
def split_dataset(source_folder, train_folder, test_folder, split_ratio=0.6):
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    files = os.listdir(source_folder)
    random.shuffle(files)
    num_train = int(len(files) * split_ratio)
    for file in files[:num_train]:
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(train_folder, file)
        shutil.copy(src_file, dst_file)
    print("Carpeta de entrenamiento creada con exito")

    for file in files[num_train:]:
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(test_folder, file)
        shutil.copy(src_file, dst_file)
    print("Carpeta de pruebas creada con exito")



In [8]:
def resize_images_in_folder(folder_name, target_size):
    
    datagen = ImageDataGenerator()
    datagen_config = {"directory": folder_name, "target_size": target_size}
    image_generator = datagen.flow_from_directory(**datagen_config, shuffle=False)
    
    for i in range(len(image_generator)):
        batch_images, batch_labels = image_generator[i]
        for j, image in enumerate(batch_images):
            if image.shape[:2] != target_size:
                resized_image = datagen.load_img(image_generator.filepaths[i * image_generator.batch_size + j], target_size=target_size)
                datagen.save_img(image_generator.filepaths[i * image_generator.batch_size + j], resized_image)

*Las dos funciones anteriores preparan las imagenes para ser trabajadas como prueba y testeo de acuerdo a los parametros que se le entreguen a las funciones. De igual manera, se trabaja una función para asignar un tamaño estandar a las imagenes de las carpetas que se especifique*

In [9]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print("GPU disponible para usar.")
else:
    print("GPU no disponible")

GPU no disponible


In [10]:
resize_images_in_folder("Data",(128,128))

Found 5631 images belonging to 4 classes.


*Teniendo todas las imagenes con el tamaño solicitado, queda realizar las divisiones de los dataset de imagenes en training y test*

In [11]:
folders = ["cloudy", "desert", "green_area", "water"]
split_ratio = 0.6

for folder in folders:
    source_folder = os.path.join("Data", folder)
    train_folder = os.path.join("Data", folder + "_train")
    test_folder = os.path.join("Data", folder + "_test")
    split_dataset(source_folder, train_folder, test_folder, split_ratio)

print("carpetas de entrenamiento y pruebas creadas")

Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
carpetas de entrenamiento y pruebas creadas


*Teniendo las imagenes separadas en test y train se puede continuar con la definición de las redes CNN*

In [21]:
train_path_c = Path('Data/cloudy_train/')
test_path_c = Path('Data/cloudy_test/')
train_path_d = Path('Data/desert_train/')
test_path_d = Path('Data/desert_test/')
train_path_g = Path('Data/green_area_train/')
test_path_g = Path('Data/green_area_test/')
train_path_w = Path('Data/water_train/')
test_path_w = Path('Data/water_test/')


In [13]:
img_width, img_height = 128, 128
batch_size = 32

Las dimensiones anteriores son siguiendo las recomendaciones del profesor

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Rotacion entre -20 y 20 grados
    width_shift_range=0.1,  # Shift horizontal hasta del 10%
    height_shift_range=0.1, # Shift vertical hasta dele 10%
    shear_range=0.2,        
    zoom_range=0.2,         # Random zoom hasta del 20%
    horizontal_flip=True,   # voltear la imagen de manera horizontal
    fill_mode='nearest'     # Llenar los nuevos pixeles creados
)

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [22]:
if train_path_c.exists():
    archivos = os.listdir(train_path_c)
    print(f'Dentro de la carpeta {str(train_path_c)} se encuentran {len(archivos)} archivos')
    train_generator_c = train_datagen.flow_from_directory(
        train_path_c,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
    )
else:
    print('Ruta no existe')
    

    
if train_path_d.exists():
    archivos = os.listdir(train_path_c)
    print(f'Dentro de la carpeta {str(train_path_d)} se encuentran {len(archivos)} archivos')
    train_generator_d = train_datagen.flow_from_directory(
        train_path_d,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
    )
else:
    print('Ruta no existe')

if train_path_g.exists():
    archivos = os.listdir(train_path_c)
    print(f'Dentro de la carpeta {str(train_path_g)} se encuentran {len(archivos)} archivos')
    train_generator_g = train_datagen.flow_from_directory(
        train_path_g,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
    )
else:
    print('Ruta no existe')

if train_path_w.exists():
    archivos = os.listdir(train_path_c)
    print(f'Dentro de la carpeta {str(train_path_w)} se encuentran {len(archivos)} archivos')
    train_generator_w = train_datagen.flow_from_directory(
        train_path_w,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
    )
else:
    print('Ruta no existe')

Dentro de la carpeta Data\cloudy_train se encuentran 900 archivos
Found 0 images belonging to 0 classes.
Dentro de la carpeta Data\desert_train se encuentran 900 archivos
Found 0 images belonging to 0 classes.
Dentro de la carpeta Data\green_area_train se encuentran 900 archivos
Found 0 images belonging to 0 classes.
Dentro de la carpeta Data\water_train se encuentran 900 archivos
Found 0 images belonging to 0 classes.


In [23]:
test_generator_c = test_datagen.flow_from_directory(
    test_path_c,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # mantener los datos en el mismo orden en que se tienen
)

test_generator_d = test_datagen.flow_from_directory(
    test_path_d,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # mantener los datos en el mismo orden en que se tienen
)

test_generator_g = test_datagen.flow_from_directory(
    test_path_g,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # mantener los datos en el mismo orden en que se tienen
)

test_generator_w = test_datagen.flow_from_directory(
    test_path_w,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # mantener los datos en el mismo orden en que se tienen
)

Found 0 images belonging to 0 classes.


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


*Teniendo los generadores de imagenes de entrenamiento y de imagenes de prueba, se puede iniciar con el desarrollo del modelo CNN*

In [18]:
inputs = Input(shape=(img_width, img_height,3))
# El objeto de tipo input tiene la altura y ancho definidos. 
# Al ser imagenes a color se cuenta con 3 canales
x = Conv2D(32, (3, 3), activation='relu', padding="same")(inputs)
# 32- numero de filtros aplicados
# (3,3) es el tamaño del kernel de convolusion 
# Se tienen en cuenta los recomendados en clase, trabajando valores inpares
# se aplica una capa convolusional con activación reul
# la misma trabajada en clases. Se le envian los inputs
x = BatchNormalization()(x)
# Ayuda a estabilizar y acelear el proceso de la red neuronal
x = MaxPooling2D((2, 2), strides=(2,2))(x)
# Reduce la dimensionalidad y hace la red mas robusta
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(2, activation='softmax')(x)
#Trabajando con softmax nos debera de clasificar entre dos clases
#nos dira una distribución de probabilidad entre estas dos
model = Model(inputs=inputs, outputs=outputs)
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,837,058 (22.27 MB)

 Trainable params: 5,836,098 (22.26 MB)

 Non-trainable params: 960 (3.75 KB)